In [1]:
import os

import numpy as np #데이터 처리
import pandas as pd #데이터 처리
import warnings
warnings.filterwarnings('ignore')
from collections import Counter # count 용도

import matplotlib.pyplot as plt # 시각화
import seaborn as sns #시각화

import folium # 지도 관련 시각화
from folium.plugins import MarkerCluster #지도 관련 시각화
import geopy.distance #거리 계산해주는 패키지 사용

import random #데이터 샘플링
from sklearn.model_selection import GridSearchCV #모델링
from sklearn.ensemble import RandomForestRegressor #모델링
path = "Jupyter Temp/프로젝트/"
pd.set_option("display.max_rows",None)

In [2]:
import folium

In [3]:
train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "test.csv")

# 1. date 변수 변환


In [4]:
train.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,...,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,...,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,...,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.0,17.0,...,26.0,14.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0
4,4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.41260,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [5]:
inout = train[["bus_route_id","in_out","latitude","longitude","station_name"]]
inout.drop_duplicates(inplace=True)
inout.head()

,bus_route_id,in_out,latitude,longitude,station_name
0,4270000,시외,33.48990,126.49373,제주썬호텔
1,4270000,시외,33.48944,126.48508,한라병원
2,4270000,시외,33.48181,126.47352,정존마을
3,4270000,시내,33.50577,126.49252,제주국제공항(600번)
4,4270000,시내,33.25579,126.41260,중문관광단지입구


In [6]:
inout = inout.groupby(by=["station_name",'bus_route_id',"in_out"]).mean().reset_index()

In [7]:
inout.head()

,station_name,bus_route_id,in_out,latitude,longitude
0,(구)구판장,30850000,시내,33.329680,126.798050
1,(구)구판장,30860000,시내,33.329705,126.797965
2,(구)구판장,30870000,시내,33.329680,126.798050
3,(구)중문동주민센터,22540000,시내,33.251085,126.434300
4,(구)중문동주민센터,22550000,시내,33.251085,126.434300


In [8]:
jeju = folium.Map([33.4,126.55], zoom_start = 11) 

In [9]:
inout.sort_values(by=["bus_route_id","latitude","longitude"],inplace=True)

In [10]:
inout.reset_index(drop=True,inplace=True)
inout.head()

,station_name,bus_route_id,in_out,latitude,longitude
0,켄싱턴리조트악근천,4270000,시내,33.23440,126.49043
1,왕대왓,4270000,시내,33.23465,126.48241
2,서건도,4270000,시외,33.23595,126.50491
3,강정농협,4270000,시외,33.23603,126.47827
4,대포항,4270000,시외,33.23974,126.43773


In [11]:
inout_loc = inout[["latitude", "longitude"]].values.tolist() 
inout_name = inout["station_name"].values.tolist() 

In [12]:
inout_loc[:6]

[[33.2344, 126.49043],
 [33.23465, 126.48241000000002],
 [33.23595, 126.50491],
 [33.23603, 126.47827],
 [33.239740000000005, 126.43773],
 [33.23977, 126.5645]]

In [13]:
for i in range(len(inout)):
    j = i+1
    
    try:
        if inout["bus_route_id"][i] == inout["bus_route_id"][j]:
            folium.PolyLine(locations=[inout_loc[i], inout_loc[j]], color="green", smooth_factor =2).add_to(jeju)
            
    except:
        print("!")

!


In [14]:
bts = pd.read_csv(path + "bus_bts.csv")

In [15]:
bts.head()

,user_card_id,bus_route_id,vhc_id,geton_date,geton_time,geton_station_code,geton_station_name,getoff_date,getoff_time,getoff_station_code,getoff_station_name,user_category,user_count
0,1.010010e+15,23000000,149793674,2019-09-10,06:34:45,360,노형오거리,2019-09-10,07:10:31,592.0,화북초등학교,1,1
1,1.010010e+15,23000000,149793674,2019-09-10,06:34:58,360,노형오거리,2019-09-10,06:56:27,3273.0,고산동산(광양방면),1,1
2,1.019160e+15,21420000,149793535,2019-09-10,07:19:07,2495,동광환승정류장4(제주방면),2019-09-10,07:40:29,431.0,정존마을,4,1
3,1.019150e+15,21420000,149793512,2019-09-09,09:14:47,3282,대정환승정류장(대정읍사무소),2019-09-09,10:02:46,431.0,정존마을,1,1
4,1.010010e+15,21420000,149793512,2019-09-09,09:28:53,2820,삼정지에듀,2019-09-09,10:21:37,2972.0,제주국제공항(종점),4,1


In [16]:
bts.dtypes

user_card_id           float64
bus_route_id             int64
vhc_id                   int64
geton_date              object
geton_time              object
geton_station_code       int64
geton_station_name      object
getoff_date             object
getoff_time             object
getoff_station_code    float64
getoff_station_name     object
user_category            int64
user_count               int64
dtype: object

In [17]:
from datetime import datetime

def on2date(z):
    try:
        date = z[0]
        time = z[1]
        string = date+" "+time
        final = datetime.strptime(string, "%Y-%m-%d %H:%M:%S")
    except:
        final = np.nan
    return final

def off2date(z):
    try:
        date = z[0]
        time = z[1]
        string = date+" "+time
        final = datetime.strptime(string, "%Y-%m-%d %H:%M:%S")
    except:
        final = np.nan
    return final

In [18]:
bts["geton"] = list(zip(bts["geton_date"],bts["geton_time"]))
bts["getoff"] = list(zip(bts["getoff_date"],bts["getoff_time"]))

bts["geton"] = bts["geton"].apply(lambda x : on2date(x))
bts["getoff"] = bts["getoff"].apply(lambda x : off2date(x))

In [19]:
bts.head()

,user_card_id,bus_route_id,vhc_id,geton_date,geton_time,geton_station_code,geton_station_name,getoff_date,getoff_time,getoff_station_code,getoff_station_name,user_category,user_count,geton,getoff
0,1.010010e+15,23000000,149793674,2019-09-10,06:34:45,360,노형오거리,2019-09-10,07:10:31,592.0,화북초등학교,1,1,2019-09-10 06:34:45,2019-09-10 07:10:31
1,1.010010e+15,23000000,149793674,2019-09-10,06:34:58,360,노형오거리,2019-09-10,06:56:27,3273.0,고산동산(광양방면),1,1,2019-09-10 06:34:58,2019-09-10 06:56:27
2,1.019160e+15,21420000,149793535,2019-09-10,07:19:07,2495,동광환승정류장4(제주방면),2019-09-10,07:40:29,431.0,정존마을,4,1,2019-09-10 07:19:07,2019-09-10 07:40:29
3,1.019150e+15,21420000,149793512,2019-09-09,09:14:47,3282,대정환승정류장(대정읍사무소),2019-09-09,10:02:46,431.0,정존마을,1,1,2019-09-09 09:14:47,2019-09-09 10:02:46
4,1.010010e+15,21420000,149793512,2019-09-09,09:28:53,2820,삼정지에듀,2019-09-09,10:21:37,2972.0,제주국제공항(종점),4,1,2019-09-09 09:28:53,2019-09-09 10:21:37


In [27]:
bts["onbus"] = list(zip(bts["geton"],bts["getoff"]))
bts["onbus"] = bts["onbus"].apply(lambda x : x[1] - x[0])

In [28]:
bts.head()

,user_card_id,bus_route_id,vhc_id,geton_date,geton_time,geton_station_code,geton_station_name,getoff_date,getoff_time,getoff_station_code,getoff_station_name,user_category,user_count,geton,getoff,하루시작,onbus
0,1.010010e+15,23000000,149793674,2019-09-10,06:34:45,360,노형오거리,2019-09-10,07:10:31,592.0,화북초등학교,1,1,2019-09-10 06:34:45,2019-09-10 07:10:31,0.0,0 days 00:35:46
1,1.010010e+15,23000000,149793674,2019-09-10,06:34:58,360,노형오거리,2019-09-10,06:56:27,3273.0,고산동산(광양방면),1,1,2019-09-10 06:34:58,2019-09-10 06:56:27,0.0,0 days 00:21:29
2,1.019160e+15,21420000,149793535,2019-09-10,07:19:07,2495,동광환승정류장4(제주방면),2019-09-10,07:40:29,431.0,정존마을,4,1,2019-09-10 07:19:07,2019-09-10 07:40:29,0.0,0 days 00:21:22
3,1.019150e+15,21420000,149793512,2019-09-09,09:14:47,3282,대정환승정류장(대정읍사무소),2019-09-09,10:02:46,431.0,정존마을,1,1,2019-09-09 09:14:47,2019-09-09 10:02:46,0.0,0 days 00:47:59
4,1.010010e+15,21420000,149793512,2019-09-09,09:28:53,2820,삼정지에듀,2019-09-09,10:21:37,2972.0,제주국제공항(종점),4,1,2019-09-09 09:28:53,2019-09-09 10:21:37,0.0,0 days 00:52:44


In [30]:
#버스에 가장 오래/적게 탄 사람
print(min(bts["onbus"]))
print(max(bts["onbus"]))

0 days 00:00:06
0 days 03:59:46


In [34]:
#버스를 가장 여러번 탄사람
bts.groupby(by="user_card_id")["bus_route_id"].count().max()

184

In [35]:
bts.groupby(by="user_card_id")["bus_route_id"].count().min()

1

In [40]:
#사람들이 가장 많이 탄 버스정류장
bts.groupby(by="geton_station_name")["bus_route_id"].count().sort_values(ascending=False).head()

geton_station_name
한라병원          50157
제주시외버스터미널     37600
제주버스터미널       31425
제주도청신제주로터리    31268
제주시청(아라방면)    29913
Name: bus_route_id, dtype: int64

In [43]:
#사람들이 가장 많이 내린 버스정류장
bts.groupby(by="getoff_station_name")["bus_route_id"].count().sort_values(ascending=False).head()

getoff_station_name
제주국제공항(종점)     42319
한라병원           38282
제주버스터미널        33966
제주시청(아라방면)     25793
제주버스터미널(종점)    23384
Name: bus_route_id, dtype: int64